In [1]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd

/Users/ash/Documents/Workspaces/PhD/Seapopym-optimisation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load DATA


In [2]:
LATITUDE = 50
LONGITUDE = -145
START_TIME = "1998-01-01"
END_TIME = "2020-02-29"
DELTA = 0.5

print(f"Latitude: {LATITUDE}\nLongitude: {LONGITUDE}\nStart Time: {START_TIME}\nEnd Time: {END_TIME}\nDelta: {DELTA}")

Latitude: 50
Longitude: -145
Start Time: 1998-01-01
End Time: 2020-02-29
Delta: 0.5


### CMEMS


In [3]:
FROM_API = True

In [4]:
# copernicusmarine.login()

coordinates = {
    "minimum_latitude": LATITUDE - DELTA,
    "maximum_latitude": LATITUDE + DELTA,
    "minimum_longitude": LONGITUDE - DELTA,
    "maximum_longitude": LONGITUDE + DELTA,
    "start_datetime": START_TIME,
    "end_datetime": END_TIME,
}


if FROM_API:
    bio = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i", **coordinates)
    bio.load()
    physic = copernicusmarine.open_dataset(dataset_id="cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i", **coordinates)
    physic.load()
else:
    bio = xr.load_dataset("...")
    physic = xr.load_dataset("...")

INFO - 2024-08-16T09:32:21Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-08-16T09:32:21Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-08-16T09:32:22Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-08-16T09:32:43Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-08-16T09:32:43Z - Dataset part was not specified, the first one was selected: "default"
INFO - 2024-08-16T09:32:44Z - Service was not specified, the default one was selected: "arco-time-series"


In [5]:
bio_sliced = bio.sel(time=slice(START_TIME, END_TIME)).sel(latitude=[LATITUDE], longitude=[LONGITUDE], method="nearest")
physic_sliced = physic.sel(time=slice(START_TIME, END_TIME)).sel(
    latitude=[LATITUDE], longitude=[LONGITUDE], method="nearest"
)
lat_attrs, lon_attrs = bio_sliced.latitude.attrs, bio_sliced.longitude.attrs
bio_sliced = bio_sliced.assign_coords(longitude=[LONGITUDE], latitude=[LATITUDE])
physic_sliced = physic_sliced.assign_coords(longitude=[LONGITUDE], latitude=[LATITUDE])
papa_data = xr.merge([bio_sliced, physic_sliced])
papa_data.latitude.attrs, papa_data.longitude.attrs = lat_attrs, lon_attrs
papa_data["time"].attrs = {"long_name": "Time", "standard_name": "time", "axis": "T"}
papa_data

<xarray.Dataset> Size: 1MB
Dimensions:              (time: 8095, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * time                 (time) datetime64[ns] 65kB 1998-01-01 ... 2020-02-29
  * longitude            (longitude) int64 8B -145
  * latitude             (latitude) int64 8B 50
  * depth                (depth) int32 12B 1 2 3
Data variables: (12/13)
    mnkc_epi             (time, latitude, longitude) float64 65kB 1.835 ... 2...
    mnkc_hmlmeso         (time, latitude, longitude) float64 65kB 2.382 ... 3...
    mnkc_lmeso           (time, latitude, longitude) float64 65kB 2.842 ... 3...
    mnkc_mlmeso          (time, latitude, longitude) float64 65kB 2.319 ... 2.62
    mnkc_mumeso          (time, latitude, longitude) float64 65kB 2.779 ... 4...
    mnkc_umeso           (time, latitude, longitude) float64 65kB 1.68 ... 2.135
    ...                   ...
    zeu                  (time, latitude, longitude) float64 65kB 57.33 ... 6...
    zooc                 (time, latitude, longitude) float64 65kB 1.618 ... 2...
    T                    (time, depth, latitude, longitude) float64 194kB 6.2...
    U                    (time, depth, latitude, longitude) float64 194kB 0.0...
    V                    (time, depth, latitude, longitude) float64 194kB 0.0...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 194kB 86....
Attributes: (12/13)
    Conventions:                      CF-1.7
    date_field:                       1998-01-01
    domain:                           global
    history:                          Created on 2022-09-28
    institution:                      CLS
    net_primary_production_model:     VGPM
    ...                               ...
    source:                           SEAPODYM-LMTL 3.0.0
    source_biogeochemical_variables:  OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS...
    source_physical_variables:        GLOBAL_REANALYSIS_PHY_001_030 CMEMS pro...
    spatial_resolution:               0.083x0.083
    temporal_resolution:              1 day
    title:                            Global ocean low and mid trophic levels...

## Gathering Data & Save


In [6]:
papa_data.to_zarr("papa_cmems.zarr", mode="w")